In [ ]:
#!sudo apt update && sudo apt install ffmpeg
#!sudo apt-get install libportaudio2 portaudio19-dev
#!sudo apt install python3-gi python3-gi-cairo gir1.2-gobject-2.0 gir1.2-gstreamer-1.0 gstreamer1.0-plugins-good gstreamer1.0-plugins-base gstreamer1.0-gl

In [ ]:
!pip install openai-whisper sounddevice scipy ollama pyttsx3 numpy

In [ ]:
!pip install edge-tts playsound==1.2.2

In [1]:
!pip install pydub

In [9]:
import sounddevice as sd
import numpy as np
import scipy.io.wavfile as wav
import whisper
import ollama
# import pyttsx3 # Ya no usamos pyttsx3
import edge_tts # ¡Nuevo!
from playsound import playsound # ¡Nuevo!
import os
import time
import sys
import asyncio # ¡Nuevo!
from pydub import AudioSegment

import nest_asyncio
nest_asyncio.apply()

In [12]:
# --- Configuración ---
SAMPLE_RATE = 16000
RECORD_SECONDS = 5
SILENCE_THRESHOLD = 0.01
WHISPER_MODEL = "base"
OLLAMA_MODEL = "llama3.2:1b" # O "llama3.2:1b" si lo tienes
TEMP_AUDIO_FILE_REC = "temp_recording.wav"
TEMP_AUDIO_FILE_TTS = "temp_tts_output.mp3" # Archivo para la salida de edge-tts
EXIT_KEYWORDS = ["adiós", "terminar", "salir", "bye", "exit"]
MICROPHONE_DEVICE_INDEX = None # Ajusta si es necesario
# --- Configuración de Voz TTS (edge-tts) ---
# Elige una voz de la lista obtenida con 'edge-tts --list-voices'
# Ejemplos: "es-ES-AlvaroNeural", "es-ES-ElviraNeural", "es-MX-JorgeNeural"
TTS_VOICE = "es-ES-ElviraNeural"

In [ ]:
# --- Listar Dispositivos de Audio (para diagnóstico) ---
# (Mantenemos la lógica de detección de micrófono de la versión anterior)
print("Buscando dispositivos de audio...")
try:
    print(sd.query_devices())
    default_input_device = sd.default.device[0]
    print(f"Dispositivo de entrada predeterminado del sistema: {default_input_device}")
    if MICROPHONE_DEVICE_INDEX is None:
        MICROPHONE_DEVICE_INDEX = default_input_device
    print(f"Usando dispositivo de entrada con índice: {MICROPHONE_DEVICE_INDEX}")
except Exception as e:
    print(f"Error al consultar dispositivos de audio: {e}. Usando predeterminado si es posible.")
    # No salimos, intentaremos continuar con el predeterminado si falla la consulta

Buscando dispositivos de audio...
   0 HDA NVidia: HDMI 0 (hw:0,3), ALSA (0 in, 8 out)
   1 HDA NVidia: HDMI 1 (hw:0,7), ALSA (0 in, 8 out)
   2 HDA NVidia: HDMI 2 (hw:0,8), ALSA (0 in, 8 out)
   3 HDA NVidia: HDMI 3 (hw:0,9), ALSA (0 in, 8 out)
   4 sof-hda-dsp: - (hw:1,0), ALSA (2 in, 2 out)
   5 sof-hda-dsp: - (hw:1,3), ALSA (0 in, 2 out)
   6 sof-hda-dsp: - (hw:1,4), ALSA (0 in, 2 out)
   7 sof-hda-dsp: - (hw:1,5), ALSA (0 in, 2 out)
   8 sof-hda-dsp: - (hw:1,6), ALSA (2 in, 0 out)
   9 sof-hda-dsp: - (hw:1,7), ALSA (2 in, 0 out)
  10 hdmi, ALSA (0 in, 8 out)
  11 pulse, ALSA (32 in, 32 out)
* 12 default, ALSA (32 in, 32 out)
Dispositivo de entrada predeterminado del sistema: 12
Usando dispositivo de entrada con índice: 12


In [14]:
# --- Inicialización Whisper ---
print("Cargando modelo Whisper...")
try:
    whisper_model = whisper.load_model(WHISPER_MODEL)
    print(f"Modelo Whisper '{WHISPER_MODEL}' cargado.")
except Exception as e:
    print(f"Error cargando el modelo Whisper: {e}")
    sys.exit(1)

# --- Inicialización TTS (edge-tts) ---
# No hay inicialización explícita como en pyttsx3, se hace al usarlo.
print(f"Configurado para usar la voz TTS: {TTS_VOICE} (requiere internet)")

Cargando modelo Whisper...
Modelo Whisper 'base' cargado.
Configurado para usar la voz TTS: es-ES-ElviraNeural (requiere internet)


In [15]:
# --- Funciones (algunas ahora son async) ---

def record_audio(filename, duration, samplerate, device_index):
    """Graba audio del micrófono especificado y lo guarda en un archivo WAV."""
    # Esta función no necesita ser async
    print(f"\nComenzando grabación desde dispositivo {device_index} ({duration} segundos)... Habla ahora.")
    try:
        recording = sd.rec(int(duration * samplerate), samplerate=samplerate, channels=1, dtype='int16', device=device_index)
        sd.wait()
        print("Grabación finalizada.")
        wav.write(filename, samplerate, recording)
        if np.abs(recording).mean() < SILENCE_THRESHOLD * np.iinfo(recording.dtype).max:
            print("Advertencia: Se detectó silencio o audio muy bajo.")
            return False
        return True
    except Exception as e:
        print(f"Error durante la grabación: {e}")
        print("Verifica el índice del dispositivo y los permisos.")
        return None

def transcribe_audio(filename, model):
    """Transcribe el archivo de audio usando Whisper."""
    # Esta función no necesita ser async
    if not os.path.exists(filename):
        print(f"Error: El archivo de audio {filename} no existe.")
        return None
    print("Transcribiendo audio...")
    try:
        # Forzar español puede ayudar a Whisper, y es coherente con la voz TTS
        result = model.transcribe(filename, fp16=False, language='es')
        transcription = result["text"].strip()
        print(f"Texto reconocido: '{transcription}'")
        # Limpiar archivo temporal de grabación AQUÍ, después de transcribir
        if os.path.exists(filename):
             try:
                 os.remove(filename)
             except Exception as e_del:
                 print(f"Advertencia: No se pudo eliminar {filename}: {e_del}")
        return transcription
    except Exception as e:
        print(f"Error durante la transcripción: {e}")
        return None

def get_llm_response(prompt, model_name, conversation_history):
    """Obtiene una respuesta del modelo Ollama manteniendo el historial."""
    # Esta función no necesita ser async
    print(f"Enviando a Ollama (modelo: {model_name})...")
    try:
        conversation_history.append({'role': 'user', 'content': prompt})
        response = ollama.chat(
            model=model_name,
            messages=conversation_history
        )
        llm_response = response['message']['content']
        conversation_history.append({'role': 'assistant', 'content': llm_response})
        print(f"Respuesta de Ollama: '{llm_response}'")
        return llm_response
    except Exception as e:
        print(f"Error contactando con Ollama: {e}")
        if conversation_history and conversation_history[-1]['role'] == 'user':
            conversation_history.pop()
        return "Lo siento, no pude procesar tu solicitud en este momento."

async def speak_text_edge(text, voice, output_filename):
    """Genera el audio TTS usando edge-tts y lo reproduce con pydub + sounddevice."""
    print("Generando voz con edge-tts...")
    try:
        # 1. Generar el archivo MP3 con edge-tts
        communicate = edge_tts.Communicate(text, voice)
        await communicate.save(output_filename)

        print("Reproduciendo respuesta con pydub + sounddevice...")

        # 2. Cargar el MP3 usando pydub
        # Asegúrate de que ffmpeg esté instalado y en el PATH
        audio = AudioSegment.from_mp3(output_filename)

        # 3. Obtener los datos de audio como un array numpy
        # Convertir a un tipo de dato que sounddevice entienda bien (float32 o int16)
        # Pydub usa int16 internamente para MP3 estándar
        samples = np.array(audio.get_array_of_samples())

        # Asegurarse de que es la forma correcta para sounddevice (N_muestras, N_canales)
        if audio.channels > 1:
            samples = samples.reshape((-1, audio.channels))
        # else: # Si es mono, puede necesitar ser (N_muestras, 1) o simplemente (N_muestras,)
              # sounddevice suele manejar bien arrays 1D para mono.
              # samples = samples.reshape((-1, 1)) # Descomentar si da error de forma

        # 4. Reproducir usando sounddevice
        sd.play(samples, audio.frame_rate, blocking=True)
        # Alternativa si blocking=True no funciona bien con asyncio:
        # sd.play(samples, audio.frame_rate)
        # sd.wait() # Espera a que termine la reproducción

        print("Reproducción finalizada.")

    except FileNotFoundError:
         print(f"Error: No se pudo encontrar ffmpeg. Asegúrate de que esté instalado y en el PATH del sistema.")
         print("Puedes instalarlo con 'sudo apt install ffmpeg' o 'sudo dnf install ffmpeg'.")
    except edge_tts.exceptions.NoAudioReceived:
        print("Error: No se recibió audio de edge-tts. Verifica la conexión a internet y la voz seleccionada.")
    except Exception as e:
        print(f"Error durante la síntesis o reproducción de voz: {e}")
    finally:
        # 5. Limpiar el archivo de audio TTS temporal (importante)
        if os.path.exists(output_filename):
            try:
                os.remove(output_filename)
            except Exception as e:
                print(f"Advertencia: No se pudo eliminar {output_filename}: {e}")


In [16]:
# --- Bucle Principal Asíncrono ---
async def main_loop():
    conversation_history = []
    print("\n--- Asistente de Voz (con edge-tts) Iniciado ---")
    print(f"Usando voz: {TTS_VOICE}")
    print(f"Di una de las siguientes palabras para salir: {', '.join(EXIT_KEYWORDS)}")
    print("Recuerda presionar Enter antes de cada vez que quieras hablar.")

    while True:
        # Usamos run_in_executor para manejar input() síncrono en un loop async
        # O simplemente podemos dejar input() como está si no causa problemas
        await asyncio.to_thread(input, "\nPresiona Enter para comenzar a grabar...")


        # 1. Grabar audio (síncrono)
        recording_result = record_audio(TEMP_AUDIO_FILE_REC, RECORD_SECONDS, SAMPLE_RATE, MICROPHONE_DEVICE_INDEX)

        if recording_result is None:
             # Error de grabación, ya se imprimió mensaje en la función
             # Podríamos querer decir algo aquí también
             await speak_text_edge("Hubo un problema con la grabación.", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
             continue
        elif not recording_result: # Silencio detectado
            retry_input = await asyncio.to_thread(input, "No detecté sonido claro. ¿Quieres intentarlo de nuevo? (s/n): ")
            if retry_input.lower() != 's':
                 await speak_text_edge("Entendido, terminando la sesión.", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
                 break
            else:
                 # No es necesario borrar aquí, transcribe_audio lo hará si existe
                 continue

        # 2. Transcribir audio (síncrono, pero la función limpia el archivo REC)
        user_text = transcribe_audio(TEMP_AUDIO_FILE_REC, whisper_model)

        if not user_text:
            await speak_text_edge("No pude entender lo que dijiste. Por favor, inténtalo de nuevo.", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
            continue

        # 3. Comprobar salida
        if any(keyword in user_text.lower() for keyword in EXIT_KEYWORDS):
            print("Detectada palabra clave de salida.")
            await speak_text_edge("Entendido. ¡Hasta luego!", TTS_VOICE, TEMP_AUDIO_FILE_TTS)
            break

        # 4. Obtener respuesta del LLM (síncrono)
        ai_response = get_llm_response(user_text, OLLAMA_MODEL, conversation_history)

        # 5. Decir la respuesta (asíncrono)
        await speak_text_edge(ai_response, TTS_VOICE, TEMP_AUDIO_FILE_TTS)

        # Pequeña pausa asíncrona
        await asyncio.sleep(0.5)

In [17]:
try:
    # Ejecuta directamente la corutina en el bucle existente
    await main_loop()
except KeyboardInterrupt:
    print("\nInterrupción por teclado detectada. Saliendo...")
finally:
    # Limpieza final de archivos temporales si aún existen
    if os.path.exists(TEMP_AUDIO_FILE_REC):
        try: os.remove(TEMP_AUDIO_FILE_REC)
        except Exception: pass
    if os.path.exists(TEMP_AUDIO_FILE_TTS):
        try: os.remove(TEMP_AUDIO_FILE_TTS)
        except Exception: pass
    print("--- Asistente de Voz Terminado ---")


--- Asistente de Voz (con edge-tts) Iniciado ---
Usando voz: es-ES-ElviraNeural
Di una de las siguientes palabras para salir: adiós, terminar, salir, bye, exit
Recuerda presionar Enter antes de cada vez que quieras hablar.

Comenzando grabación desde dispositivo 12 (5 segundos)... Habla ahora.
Grabación finalizada.
Transcribiendo audio...
Texto reconocido: 'Hola, ¿cómo estás?'
Enviando a Ollama (modelo: llama3.2:1b)...
Respuesta de Ollama: 'Estoy bien, gracias. ¿Y tú? ¿En qué puedo ayudarte hoy?'
Generando voz con edge-tts...
Reproduciendo respuesta con pydub + sounddevice...
Reproducción finalizada.

Comenzando grabación desde dispositivo 12 (5 segundos)... Habla ahora.
Grabación finalizada.
Transcribiendo audio...
Texto reconocido: 'Adiós'
Detectada palabra clave de salida.
Generando voz con edge-tts...
Reproduciendo respuesta con pydub + sounddevice...
Reproducción finalizada.
--- Asistente de Voz Terminado ---
